In [1]:
import mlflow
from ultralytics import YOLO
import torch


#mlflow.set_tracking_uri("http://13.43.219.126:8080")

In [2]:
torch.cuda.is_available()

True

In [3]:
!yolo setting datasets_dir=$PWD

💡 Learn about settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
Printing '/home/xecarlox/.config/Ultralytics/settings.yaml'

settings_version: 0.0.4
datasets_dir: /home/xecarlox/Desktop/Projects/bottle_recognition
weights_dir: weights
runs_dir: runs
uuid: bd7c67341394cbb91947d38f2f24be674e49cb1532e2d13cdc3d37ffe9400404
sync: true
api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: true



In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="llgtjSRbLz3hdjjaUEfb")
project = rf.workspace("erkan-unal").project("detect-o9dby")
dataset = project.version(1).download("yolov8")

!sed -i "s/detect-1/../g" detect-1/data.yaml

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.219, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to detect-1 in yolov8:: 100%|███████████████████████████████████████| 29430/29430 [00:01<00:00, 25497.37it/s]


In [5]:
model = YOLO('yolov8n.pt')

In [6]:
dataset_yaml = dataset.location+'/data.yaml'


In [7]:
with mlflow.start_run() as run: 
    model.train(data=dataset_yaml, epochs=1, imgsz=640, device=0, batch=8)
    print(run.info.run_id)

Ultralytics YOLOv8.0.219 🚀 Python-3.8.18 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7974MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/xecarlox/Desktop/Projects/bottle_recognition/detect-1/data.yaml, epochs=1, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /home/xecarlox/Desktop/Projects/bottle_recognition/detect-1/train/labels.cache... 10515 images, 929 backgrounds, 0 corrupt: 

train: WARNING ⚠️ /home/xecarlox/Desktop/Projects/bottle_recognition/detect-1/train/images/02_03_2020_14_48_39_jpeg.rf.e4a248f5cb036571cb56bfb8c6923da2.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/xecarlox/Desktop/Projects/bottle_recognition/detect-1/train/images/plasticbottle81_jpeg.rf.f7289a7e2c79a7fb20c5b977ad39e5c6.jpg: 1 duplicate labels removed
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 4094, len(boxes) = 11605. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /home/xecarlox/Desktop/Projects/bottle_recognition/detect-1/valid/labels.cache... 2557 images, 189 backgrounds, 0 corrupt: 100

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1186, len(boxes) = 2752. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

2023/11/28 16:40:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


MLflow: logging run_id(9495591a3b7048e498d8f41472ab52ea) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
MLflow: WARNING ⚠️ Failed to initialize: Run '9495591a3b7048e498d8f41472ab52ea' not found
MLflow: WARNING ⚠️ Not tracking this run
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train15
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      1.23G      1.151      2.326      1.494          5        640: 100%|██████████| 1315/1315 [01:15<00:00, 17.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 160/160 [00:09<00:00, 17.08it/s]


                   all       2557       2752      0.367      0.402      0.348      0.191


MlflowException: Run '9495591a3b7048e498d8f41472ab52ea' not found

In [ ]:
experiment_name = "experiment-1"
current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
experiment_id=current_experiment['experiment_id']

In [ ]:
run_id=run.info.run_id
run.info

In [ ]:
run.to_dictionary()

In [ ]:
model_name='yolov8-trial2'

In [ ]:
# The default path where the MLflow autologging function stores the model
artifact_path = "runs/detect/train22"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)
 
model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

In [ ]:
mlflow.pytorch.log_model()

In [ ]:
from mlflow.models import infer_signature

In [ ]:
signature = infer_signature(X_train, lr.predict(X_train))

In [ ]:
model = mlflow.pyfunc.load_model("mlflow-artifacts:/1/b14003bb127b4c07b091f264a56bf47b/artifacts", dst_path='/home/coena98/bottle_recognition/LoadModel')
print(model._model_meta._signature)

In [ ]:
client = MlflowClient(mlflow.get_tracking_uri()

model_uri = client.get_model_version_download_uri('yolov8-trial2','10')
model_info = mlflow.models.get_model_info(model_uri)
model_info._signature_dict